### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta, date
import pytz
import re
import string
import demoji
from nltk.tokenize import regexp_tokenize, word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [2]:
# Setting option to display all columns for dataframes
pd.set_option('display.max_columns', None)

### Import data from tweets

Let's start by defining a function to import the tweets from the csv files we did on the data aquisition, converting timezone from UTC to ET and adding additional columns for date, hour, minutes and day of the week.

In [3]:
def load_tweets(stock):
    '''Function to import tweets from csv files, converts timezone from UTC to ET, and adds additional columns for date, 
    hour, minutes and day of week'''
    
    # Read csv files for years 2018-2022 and concat them together resetting the index
    df = pd.read_csv(f'../data/raw/{stock}_tweets_all.csv', index_col=0, parse_dates=['datetime'])
    
    # Convert timezone from UTC to ET
    df['datetime'] = df['datetime'].apply(lambda x: x.astimezone('US/Eastern'))
    
    # Create column with stock ticker
    df['stock'] = stock.upper()
    
    # Get date and hour from datetime column, and add day of the week to filter weekends
    df['date'] = df['datetime'].apply(lambda x: x.date())
    df['hour'] = df['datetime'].apply(lambda x: x.hour)
    df['minutes'] = df['datetime'].apply(lambda x: x.minute)
    df['dayofweek'] = df['datetime'].apply(lambda x: x.dayofweek)
    
    print(f"Number of {stock} tweets loaded: {df.shape[0]}")
    
    return df

Let's use our function to pull the tweets

In [4]:
googl_df = load_tweets('googl')
xom_df = load_tweets('xom')
jpm_df = load_tweets('jpm')

Number of googl tweets loaded: 118420
Number of xom tweets loaded: 42956
Number of jpm tweets loaded: 57913


Consolidating all tweets in one single dataframe

In [5]:
tweets_df = pd.concat([googl_df, xom_df, jpm_df], ignore_index=True)

In [6]:
tweets_df.sample(5)

,datetime,text,username,likeCount,replyCount,retweetCount,stock,date,hour,minutes,dayofweek
171599,2019-10-16 00:28:33-04:00,#Watchlist results10/15 $AAPL $ABBV $ADBE $AMA...,TheFireguy343,1,0,0,JPM,2019-10-16,0,28,2
160356,2021-02-02 07:48:02-05:00,"A Market No-Doubter, Vaccine Milestone, GDP Gr...",Sarge986,10,4,10,XOM,2021-02-02,7,48,1
31966,2019-04-19 10:54:45-04:00,@saxena_puru Totally agree. I consider $AMZN $...,Matt_Cochrane7,8,3,0,GOOGL,2019-04-19,10,54,4
143199,2021-06-16 10:57:14-04:00,@Convertbond Bingo! Like when they removed $xo...,Kingpin_15,2,0,0,XOM,2021-06-16,10,57,2
106463,2021-06-06 10:52:58-04:00,$BB Press Release: Investor Conference this We...,_terrig,1,0,0,GOOGL,2021-06-06,10,52,6


In [7]:
tweets_df.shape

(219289, 11)

<br> Let's define a function that will tell us what date to pull from the historical stock data.<br>
<br> If the tweet is after 4:00pm, it will set prediction date to next day.
<br> If the tweet is before 9:30am, it will set prediction date for that same day.
<br> If the tweet is from the weekend, it will set prediction date to next Monday.
<br> Otherwise it will label as 'market hours'

In [8]:
def get_prediction_date(date, hour, minutes, dayofweek):
    '''Function that returns the prediction date or it states that these are market hours'''
    
    # Monday - Thursday 
    # After 4pm: return next day, Before 7:30am return same day, Else return market hours
    if dayofweek in [0,1,2,3]:
        if hour >= 16:
            return (date + timedelta(days=1))
        elif ((hour < 7) & (minutes < 60)) | ((hour == 7) & (minutes < 30)):
            return date
        else:
            return 'market hours'
    
    # Friday
    # After 4pm: return next Monday, Before 7:30am return same day, Else return market hours
    if dayofweek == 4:
        if hour >= 16:
            return (date + timedelta(days=3))
        elif ((hour < 7) & (minutes < 60)) | ((hour == 7) & (minutes < 30)):
            return date
        else:
            return 'market hours'
        
    # Saturday
    # Regardless of hour, return next Monday
    if dayofweek == 5:
        return (date + timedelta(days=2))
    
    # Sunday
    # Regardless of hour, return next Monday
    if dayofweek == 6:
        return (date + timedelta(days=1))

In [9]:
tweets_df['predictiondate'] = tweets_df.apply(lambda x: get_prediction_date(x.date, x.hour, x.minutes, x.dayofweek), axis=1)

In [10]:
tweets_df.sample(5)

,datetime,text,username,likeCount,replyCount,retweetCount,stock,date,hour,minutes,dayofweek,predictiondate
91500,2021-12-07 09:36:04-05:00,$SHOP up massively too\n\n$GOOGL\n\n$AMZN\n\nb...,StockHollywood,2,0,0,GOOGL,2021-12-07,9,36,1,market hours
31408,2019-04-29 16:12:26-04:00,$GOOGL 1220 support,brianinvest,1,0,0,GOOGL,2019-04-29,16,12,0,2019-04-30
18244,2019-12-26 13:40:45-05:00,$AMZN keeps grinding higher... 1900 can happen...,EliteOptions2,10,0,1,GOOGL,2019-12-26,13,40,3,market hours
87305,2021-02-01 22:10:34-05:00,Why Ford #Stock Just Put its Pedal to the Meta...,bluerizzle_brad,1,0,1,GOOGL,2021-02-01,22,10,0,2021-02-02
153306,2021-08-06 10:59:42-04:00,🥊 5 Oil Majors Compared 🏆 \n\nAll have reporte...,European_DGI,60,9,12,XOM,2021-08-06,10,59,4,market hours


### Download stock price historical data

We'll create a function to download stock price historical data from Yahoo Finance, remove unnecessary columns, and add more columns we'll need later.
<br> The historical data will be in 1 day intervals.
<br><br> The way we're calculating the stock price change is last day Close vs today's Open, this will determine if the price went up or down.
<br> With this information we'll create our target variable 'price_movement' that will have a 1 if it goes up or a 0 if it goes down.

In [11]:
def get_stock_data(stock):
    
    # pulling data from Yahoo Finance
    df_stock = yf.download(tickers = stock, start='2018-01-01', end='2022-12-31', interval = '1d', rounding=True)  
    df_stock = df_stock.drop(columns=['High','Low','Adj Close','Volume'], axis=1)  # removing columns we won't need
    
    # Creating additional columns we'll need later
    df_stock['stock'] = stock
    df_stock['date'] = df_stock.index.date
    
    # Resetting index
    df_stock = df_stock.reset_index()
    
    # Adding column for price change
    for i in range(1, len(df_stock)):
        df_stock.loc[i,'price_change'] = round(df_stock.loc[i]['Open'] - df_stock.loc[i-1]['Close'],2)
        
    # Adding column for price change percentage
    for i in range(1, len(df_stock)):
        df_stock.loc[i,'price_change_percent'] = round( ((df_stock.loc[i]['Open'] - df_stock.loc[i-1]['Close'])\
                                                    / df_stock.loc[i-1]['Close'])*100 ,2)
    
    # Adding column for price_movement
    df_stock['price_movement'] = df_stock['price_change'].apply(lambda x: 1 if x > 0 else 0)
    
    # Dropping first row since it won't have a price change
    df_stock = df_stock[1:]
    
    return df_stock

In [12]:
googl_stock = get_stock_data('GOOGL')
xom_stock = get_stock_data('XOM')
jpm_stock = get_stock_data('JPM')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [13]:
all_stocks = pd.concat([googl_stock, xom_stock, jpm_stock], ignore_index=True)

In [14]:
all_stocks.sample(5)

,Date,Open,Close,stock,date,price_change,price_change_percent,price_movement
896,2021-07-27 00:00:00-04:00,134.25,131.90,GOOGL,2021-07-27,0.21,0.16,1
438,2019-10-01 00:00:00-04:00,61.12,60.30,GOOGL,2019-10-01,0.06,0.10,1
1392,2018-07-17 00:00:00-04:00,82.26,82.31,XOM,2018-07-17,-0.23,-0.28,0
1422,2018-08-28 00:00:00-04:00,80.58,80.28,XOM,2018-08-28,0.18,0.22,1
3228,2020-10-30 00:00:00-04:00,97.91,98.04,JPM,2020-10-30,0.74,0.76,1


In [15]:
all_stocks.shape

(3774, 8)

We will apply a threshold to the price_change_percent. We want to exclude small moves in price. We can see that if we apply a 0.3% threshold we still keep 69% of our records.

In [16]:
all_stocks[(all_stocks.price_change_percent > 0.3) | (all_stocks.price_change_percent < -0.3)].count()['Open']

2603

In [17]:
2603/3774

0.6897191308956014

In [18]:
all_stocks = all_stocks[(all_stocks.price_change_percent > 0.3) | (all_stocks.price_change_percent < -0.3)]
all_stocks.shape

(2603, 8)

### Merge data from tweets and stock price data

We'll merge the data from the tweets and stock price based on stock and the prediction date we got from our function. 
<br>This will only keep tweets after market hours and all tweets from the weekends.

In [19]:
def merge_tweet_stock(tweets_df, stocks_df):
    
    print(f'Tweets prior to merge: {tweets_df.shape[0]}')
    merged_df = pd.merge(tweets_df, stocks_df, left_on=['predictiondate','stock'], right_on=['date','stock'])
    print(f'Tweets after merge: {merged_df.shape[0]}\n')

    # Dropping columns we won't need
    merged_df = merged_df.drop(columns=['Open','Close','Date','price_change','price_change_percent','date_y'], axis=1)
       
    return merged_df

In [20]:
stocks_merged = merge_tweet_stock(tweets_df, all_stocks)

Tweets prior to merge: 219289
Tweets after merge: 72534



Our number of tweets went from 219k to 72k when we filtered out market hours, it's still a decent amount of tweets.

In [21]:
stocks_merged.sample(5)

,datetime,text,username,likeCount,replyCount,retweetCount,stock,date_x,hour,minutes,dayofweek,predictiondate,price_movement
52713,2021-02-11 16:02:26-05:00,RECAP 2/11 Unusual Puts:\n$XOM Feb 48.5 P\n$FO...,OpenOutcrier,3,1,0,XOM,2021-02-11,16,2,3,2021-02-12,0
59672,2020-11-13 16:15:55-05:00,Something fishy\n\nTons of $TLT dark pool buys...,breakingoptions,9,0,1,JPM,2020-11-13,16,15,4,2020-11-16,1
25413,2021-09-25 11:02:52-04:00,Microdisplays: The key high value (high margin...,DZierler,6,1,1,GOOGL,2021-09-25,11,2,5,2021-09-27,0
57217,2019-09-09 16:08:47-04:00,4 Stocks To Like For The Next Decade https://t...,JorelLaraKalel,1,0,0,JPM,2019-09-09,16,8,0,2019-09-10,1
17704,2020-04-28 19:11:56-04:00,I just read $GOOGL comments on their outlook. ...,MoMoBagholder,20,2,1,GOOGL,2020-04-28,19,11,1,2020-04-29,1


### Tweet text processing

Now we'll start cleaning and processing the text from the tweets.

**Converting to lower case**

In [22]:
stocks_merged['processed_text'] = stocks_merged['text'].apply(lambda x: x.lower())

In [23]:
stocks_merged['processed_text'].sample(5)

37140    roundhill's @mattdorta covers a top headline f...
4264                 $googl ruining my bear raid dreams...
23156    analysis for 11.03 (ignore the previous tweet ...
32474    @cjoppel i like $msft the best as well. $googl...
68869    @kerberos007 not quite: from $cs $ubs $jpm etc...
Name: processed_text, dtype: object

**Find stock tickers ($)**

In [24]:
stocks_merged['tickers'] = stocks_merged['processed_text'].apply(lambda x: re.findall(r'[$][A-Za-z][\S]*', x))

In [25]:
stocks_merged['number_tickers'] = stocks_merged['tickers'].apply(lambda x: len(x))

In [26]:
stocks_merged.sample(2)

,datetime,text,username,likeCount,replyCount,retweetCount,stock,date_x,hour,minutes,dayofweek,predictiondate,price_movement,processed_text,tickers,number_tickers
42208,2020-10-21 00:31:00-04:00,$SPXS $QQQ $TSLA $UBER $VXX $IEMG $SNDL $ITUB ...,TashaCl14800837,1,0,0,XOM,2020-10-21,0,31,2,2020-10-21,0,$spxs $qqq $tsla $uber $vxx $iemg $sndl $itub ...,"[$spxs, $qqq, $tsla, $uber, $vxx, $iemg, $sndl...",29
35531,2021-02-20 12:36:44-05:00,Barron’s cover.\n$ORCL after the cloud ☁️ \n\n...,commandenteSD,2,1,0,GOOGL,2021-02-20,12,36,5,2021-02-22,0,barron’s cover.\n$orcl after the cloud ☁️ \n\n...,"[$orcl, $amzn, $msft, $googl)]",4


Let's apply a filter based on number of tickers since we want specific ones with 1 ticker on the text.

In [27]:
stocks_merged = stocks_merged[stocks_merged['number_tickers'] < 2].reset_index(drop=True)
stocks_merged.shape

(16327, 16)

**Find hashtags (#)**

In [28]:
stocks_merged['hashtags'] = stocks_merged['processed_text'].apply(lambda x: re.findall(r'[#][A-Za-z][\S]*', x))

In [29]:
stocks_merged['number_hashtags'] = stocks_merged['hashtags'].apply(lambda x: len(x))

In [30]:
stocks_merged.sample(3)

,datetime,text,username,likeCount,replyCount,retweetCount,stock,date_x,hour,minutes,dayofweek,predictiondate,price_movement,processed_text,tickers,number_tickers,hashtags,number_hashtags
12263,2018-03-21 17:16:22-04:00,"""My hope is that our investment continues to h...",jpmorgan,76,15,36,JPM,2018-03-21,17,16,2,2018-03-22,0,"""my hope is that our investment continues to h...",[$jpm's],1,[],0
3431,2020-07-11 13:31:59-04:00,Se viene $GOOGL https://t.co/65vxHuA0uv,francastromt,47,1,4,GOOGL,2020-07-11,13,31,5,2020-07-13,1,se viene $googl https://t.co/65vxhua0uv,[$googl],1,[],0
1095,2018-03-11 09:44:18-04:00,$GOOGL In the gap and looking for it to fill i...,OptionsMike,11,0,0,GOOGL,2018-03-11,9,44,6,2018-03-12,1,$googl in the gap and looking for it to fill i...,[$googl],1,[],0


**Find mentions (@)**

In [31]:
stocks_merged['mentions'] = stocks_merged['processed_text'].apply(lambda x: re.findall(r'[@][A-Za-z][\S]*', x))

In [32]:
stocks_merged['number_mentions'] = stocks_merged.loc[:,'mentions'].apply(lambda x: len(x))

In [33]:
stocks_merged.sample(3)

,datetime,text,username,likeCount,replyCount,retweetCount,stock,date_x,hour,minutes,dayofweek,predictiondate,price_movement,processed_text,tickers,number_tickers,hashtags,number_hashtags,mentions,number_mentions
10688,2021-06-02 16:56:35-04:00,"$XOM another daily chart breakout, which I ove...",DrGustoUmar,1,1,0,XOM,2021-06-02,16,56,2,2021-06-03,0,"$xom another daily chart breakout, which i ove...",[$xom],1,[],0,[],0
10517,2021-07-05 16:35:46-04:00,Was just putting together the Watchlist and wa...,TradingWithZack,18,1,0,XOM,2021-07-05,16,35,0,2021-07-06,0,was just putting together the watchlist and wa...,[$xom],1,[],0,[],0
1430,2019-10-28 16:29:00-04:00,"$GOOGL still green on the day, closed Fri 1264. 🤣",WaitCapital,4,0,0,GOOGL,2019-10-28,16,29,0,2019-10-29,0,"$googl still green on the day, closed fri 1264. 🤣",[$googl],1,[],0,[],0


**Remove numbers**

In [34]:
stocks_merged['processed_text'] = stocks_merged['processed_text'].apply(lambda x: re.sub(r'\d+','', x))

In [35]:
stocks_merged['processed_text'].sample(5)

9171     $xom #xom one gap down, two to go! https://t.c...
15642    "you're going to see very good beats because o...
2900     how i see $googl (kraken) vs. the internet htt...
1443     $googl added today to the hedge fund telemetry...
5697                                           $googl beat
Name: processed_text, dtype: object

**Remove punctuation**

In [36]:
# Excluding from string.punctuation: #, @, $
punctuation_modified = '!"%&\'()*+,-./:;<=>?[\\]^_`{|}~'

In [37]:
stocks_merged['processed_text'] = stocks_merged['processed_text'].apply(lambda x: x.translate(str.maketrans('','', punctuation_modified)))

In [38]:
stocks_merged['processed_text'].sample(5)

5618                 youtube revenue growth is wow\n$googl
8806     $xom  options traders pricing in exxon dividen...
12296    did jamie dimon get a copy $jpm httpstcoylnzhmroa
4868     lazy $googl sotp \nservices  corporate costs $...
16122    $jpm has much deeper pockets than melvin and s...
Name: processed_text, dtype: object

**Convert emojis to text**

Defining a function to convert emojis to text. We'll use demoji package and we will add "_" to emojis to differentiate from regular words.

In [39]:
def convert_emojis(text):
    
    # Finding all emojis in the text
    emoji_dict = demoji.findall(text)
    
    # For every emoji replace with the text from demoji
    for item in emoji_dict:
        emoji_dict[item] = '_' + '_'.join(emoji_dict[item].split()) + '_ '
        text = text.replace(item, emoji_dict[item])
    
    return text

In [40]:
stocks_merged['processed_text'] = stocks_merged['processed_text'].apply(lambda x: convert_emojis(x))

In [41]:
stocks_merged['processed_text'].sample(5)

12235    $jpm is currently  at $ the stock remains broa...
13166                        lol the i was wrong call $jpm
10762    big news #exxonmobil loses proxy battle at agm...
9368     someone just demed me asking how i prevent tim...
423      @stockhunter aaaand i’m already making money  ...
Name: processed_text, dtype: object

**Replacing \n (new lines)**

In [42]:
stocks_merged['processed_text'] = stocks_merged['processed_text'].apply(lambda x: x.replace('\n',' '))

In [43]:
stocks_merged['processed_text'].sample(5)

8474                                            lol $xom $
6251     $googl waymo unit said to consider raising as ...
10407    $xom  exxon considers pledging ‘netzero’ carbo...
14521    scoop baseball star alex rodriguez at centre o...
11604    i just ask myself why all the bad news on $xom...
Name: processed_text, dtype: object

**Remove white spaces**

In [44]:
stocks_merged['processed_text'] = stocks_merged['processed_text'].apply(lambda x: x.strip())

In [45]:
stocks_merged['processed_text'].sample(5)

13266    $jpm settled  to $ todays close above the dma ...
1048     nailed the whole move spx puts  to  $googl put...
9362     $xom  stop worrying about an exxon mobil divid...
3334     on the $googl call  very thorough  technocrati...
826        alphabet upgraded to buy at stifel pt $  $googl
Name: processed_text, dtype: object

**Getting length of processed text (tweet)**

In [46]:
stocks_merged['text_length'] = stocks_merged['processed_text'].apply(lambda x: len(x))

In [47]:
stocks_merged['text_length'].sample(5)

3841     34
13504    57
7345     97
16180    52
12484    51
Name: text_length, dtype: int64

**Lematization**

In [48]:
lemmatizer = WordNetLemmatizer()

In [49]:
# Splitting text in tokens to use lemmatizer
stocks_merged['tokens'] = stocks_merged['processed_text'].apply(lambda x: x.split())

In [50]:
stocks_merged['lem_text'] = stocks_merged['tokens'].apply(lambda x: [lemmatizer.lemmatize(token) for token in x])

In [51]:
stocks_merged['lem_text'].sample(5)

3983     [but, but, but, he, said, $googl, had, enginee...
13651    [bank, were, left, for, dead, these, #s, are, ...
12957    [$jpm, reported, q, this, morning, missing, on...
12887    [$jpm, letting, price, develop, a, few, day, a...
10822    [mw, exxon, mobil, stock, price, target, raise...
Name: lem_text, dtype: object

In [52]:
# Converting from lemmatized tokens to lemmatized string
stocks_merged['lem_text'] = stocks_merged['lem_text'].apply(lambda x: ' '.join(x))

In [53]:
stocks_merged['lem_text'].sample(5)

547      the giant that is $googl ha finally escaped th...
9202     $xom to report earnings on may bmo with an exp...
7158     @everytimeicash yep mega cal tech within tech ...
14416    jpmorgan nyse $jpm reported q eps of $ $ bette...
12698           another earnings beat by a major bank $jpm
Name: lem_text, dtype: object

In [54]:
# Dropping tokens column since we won't need it anymore
stocks_merged = stocks_merged.drop(columns='tokens')

In [55]:
stocks_merged.sample(3)

,datetime,text,username,likeCount,replyCount,retweetCount,stock,date_x,hour,minutes,dayofweek,predictiondate,price_movement,processed_text,tickers,number_tickers,hashtags,number_hashtags,mentions,number_mentions,text_length,lem_text
13489,2020-08-24 06:00:28-04:00,Barclays hires $JPM's Esteve to head equity ca...,InvestingStockz,1,0,0,JPM,2020-08-24,6,0,0,2020-08-24,1,barclays hires $jpms esteve to head equity cap...,[$jpm's],1,[],0,[],0,87,barclays hire $jpms esteve to head equity capi...
10427,2021-07-26 17:26:27-04:00,@Kentuckyhot I bought about 14k in $XOM calls ...,jenny_kentucky,2,1,0,XOM,2021-07-26,17,26,0,2021-07-27,0,@kentuckyhot i bought about k in $xom calls to...,[$xom],1,"[#stocks, #wsb, #wallstreet, #stockstobuy, #st...",6,[@kentuckyhot],1,152,@kentuckyhot i bought about k in $xom call tod...
5627,2021-07-27 16:19:40-04:00,Flow knows best. \n\n$GOOGL 7/30 2800c taken @...,BulITrades,6,0,0,GOOGL,2021-07-27,16,19,1,2021-07-28,1,flow knows best $googl c taken @ est am thi...,[$googl],1,[],0,[],0,136,flow know best $googl c taken @ est am this mo...


## Adding Sentiment feature

We'll use roBERTa, which is a pre-trained model on tweets from HuggingFace.

https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment

In [56]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from transformers import pipeline
from scipy.special import softmax

In [57]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

sentiment_task = pipeline("sentiment-analysis", model=MODEL, tokenizer=tokenizer)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Running one example

In [58]:
sentiment_task("google stock is going up")

[{'label': 'positive', 'score': 0.8435651659965515}]

In [59]:
stocks_merged['sentiment'] = stocks_merged['lem_text'].apply(lambda x: sentiment_task(x))

In [60]:
stocks_merged.sample(3)

,datetime,text,username,likeCount,replyCount,retweetCount,stock,date_x,hour,minutes,dayofweek,predictiondate,price_movement,processed_text,tickers,number_tickers,hashtags,number_hashtags,mentions,number_mentions,text_length,lem_text,sentiment
7078,2021-03-29 20:39:12-04:00,$GOOGL #Alphabet Class A GOOGL - Embedded Expe...,ResearchPool,1,0,0,GOOGL,2021-03-29,20,39,0,2021-03-30,1,$googl #alphabet class a googl embedded expec...,[$googl],1,"[#alphabet, #equity, #stocks]",3,[],0,266,$googl #alphabet class a googl embedded expect...,"[{'label': 'neutral', 'score': 0.8904085755348..."
12530,2019-10-06 10:16:24-04:00,@RMKOutFront @saxena_puru I understand banks (...,weeklystockchar,2,0,0,JPM,2019-10-06,10,16,6,2019-10-07,0,@rmkoutfront @saxenapuru i understand banks le...,[$jpm)],1,[],0,"[@rmkoutfront, @saxena_puru]",2,292,@rmkoutfront @saxenapuru i understand bank led...,"[{'label': 'negative', 'score': 0.781817853450..."
14366,2020-01-25 10:46:48-05:00,Financials started rolling over prior to the c...,gilmoreport,8,1,1,JPM,2020-01-25,10,46,5,2020-01-27,0,financials started rolling over prior to the c...,[$jpm],1,[#eatmyshorts],1,[],0,224,financials started rolling over prior to the c...,"[{'label': 'neutral', 'score': 0.7694271802902..."


In [61]:
# Breaking sentiment taks in label and score
stocks_merged['sentiment_label'] = stocks_merged['sentiment'].apply(lambda x: x[0]['label'])
stocks_merged['sentiment_score'] = stocks_merged['sentiment'].apply(lambda x: x[0]['score'])

# Dropping original column
stocks_merged = stocks_merged.drop(columns='sentiment')

In [62]:
stocks_merged.sample(3)

,datetime,text,username,likeCount,replyCount,retweetCount,stock,date_x,hour,minutes,dayofweek,predictiondate,price_movement,processed_text,tickers,number_tickers,hashtags,number_hashtags,mentions,number_mentions,text_length,lem_text,sentiment_label,sentiment_score
12483,2019-10-19 09:47:12-04:00,Q: Will @jpmorgan CEO Jamie Dimon take a hit f...,rcwhalen,15,1,7,JPM,2019-10-19,9,47,5,2019-10-21,1,q will @jpmorgan ceo jamie dimon take a hit fo...,[$jpm],1,[],0,"[@jpmorgan, @wework, @rbrtrmstrng, @bhgreeley,...",8,270,q will @jpmorgan ceo jamie dimon take a hit fo...,negative,0.542816
16244,2021-01-15 01:17:42-05:00,$JPM\n\nWhat is Toxic Securities Abuses?\n\nOr...,JimJame74888138,2,0,1,JPM,2021-01-15,1,17,4,2021-01-15,0,$jpm what is toxic securities abuses or anti...,[$jpm],1,[],0,[],0,235,$jpm what is toxic security abuse or antimoney...,negative,0.899618
8422,2020-12-02 18:20:03-05:00,$XOM #XOM Balloon breakout today.\n\nhttps://t...,TrendSpider,17,1,0,XOM,2020-12-02,18,20,2,2020-12-03,1,$xom #xom balloon breakout today httpstcomhmk...,[$xom],1,[#xom],1,[],0,51,$xom #xom balloon breakout today httpstcomhmko...,neutral,0.670930


Let's look at the sentiment label breakdown

In [63]:
stocks_merged['sentiment_label'].value_counts(normalize=True)

neutral     0.580450
positive    0.262387
negative    0.157163
Name: sentiment_label, dtype: float64

Now let's take a look at a few examples of each label

In [64]:
pd.set_option('display.max_colwidth', 200)

In [65]:
stocks_merged[stocks_merged['sentiment_label'] == 'positive'].sample(10)[['text']]

,text
15554,Damn she is good. $JPM
1201,as for $GOOGL: it's up 40% in a yr. a 4% dip is a gift in this mkt
15535,@WealthOrDie I invested heavy into $JPM post GFC and then again London Whale debacle. I really just invested in Jaime. Glad he beat his cancer too. The guy is a warrior and one of the best CEOS ever
5390,"$GOOGL Daily Chart 🧃\n\nStill ranging in this ascending channel, I will patiently wait for the break of 2767 to go long here. Volume was decent on Friday, but we're going to need some more if we w..."
4920,$GOOGL potential weekly breakout this week. Explosive move in the making. 🍿
14634,Trading Ideas: $JPM looking for big move this week https://t.co/B7BJWXJRld 📈 FREE trade-of-week via ⟶ https://t.co/BCEHq7Ce2N https://t.co/v6zjV2U0KG
9002,Covid breakdown offering a big buying opportunity in $XOM. https://t.co/eXSx1qTw1l
6541,Monster ! $GOOGL
10164,$XOM revival of pattern - expecting higher 60 -64 tgt https://t.co/lNYDej30jH
3261,$GOOGL Cup formation. Very strong recently. 1542 was a buy when it hit above ER gap down. 1690 is the next target on the break and stay above the pink resistance line. https://t.co/T6m9je5Mx0


In [66]:
stocks_merged[stocks_merged['sentiment_label'] == 'negative'].sample(10)[['text']]

,text
16291,@IncomeSharks A $jpm coin lol. Isn't it JPM that just dodged a major scandal yet again this and still got a bailout. Hoping one day bankers actually lose
13160,@The_Real_Fly just another smack on the wrist. Anyone add up the running tab $JPM has paid in fines? Has to be staggering sum
10911,I forgot about $XOM leading too.
4424,"(CNBC) - Google has told its employees that they will lose pay — and will eventually be fired — if they don’t comply with the company’s Covid-19 vaccination policy, according to internal documents..."
713,"Not so “Quiet” anymore…but what is a “Tariff” vs a Tariff “Literally”? Devalued Yuan, higher tariffs in China, hmmm, seems shooting self in foot was not good idea #China. Come to table, work toget..."
3817,"The #GoogleMaps SDK that's utilized by thousands of apps to render Google maps, is having issues causing app crashes. Reports note crashes on Lyft, Uber, Doordash so far. $Googl @9to5Google #MapsS..."
1856,"This really is a problem:\n\nJen Gennai lays out the truth that $GOOGL is a biased, editorial company that has a specific political agenda. \n\nhttps://t.co/wExCClBLwd"
6410,Inexplicably I have never owned $GOOGL and right now does not seem like the time to get in. I’m not calling for a major crash but extended weekly RSI typically leads to consolidations. And that MA...
3335,"Advertisers on Google have suffered mass layoffs and other cutbacks during the pandemic, and marketing budgets are often the first to get slashed https://t.co/Ms2e0QSVI2 $GOOGL https://t.co/3yKDVe..."
14472,"Shut the fuck up talking about Drug dealers are destroying the community if you're a Nigga invested in companies like $JPM. I ain't saying don't invest I'm just saying stfu. \n.\n.\n.\n.\n.\n""You ..."


In [67]:
stocks_merged[stocks_merged['sentiment_label'] == 'neutral'].sample(10)[['text']]

,text
12452,What is making $JPM re-think it's balance sheet?
15192,"@bonjour247 Yes, $JPM average deposit rose from $1,636bn in 20Q1 pre-COVID to $2,225bn by 21Q1, driving the y/y growth in assets. Most of this has gone into Deposit w/ Banks &amp; Investment Secur..."
4682,@plantmath1 * Unless it's $GOOGL
798,$GOOGL calls bot when it was early 1020s. around 27. Now 54. 1100 resistance. Lets C.
14461,"15:08:14 12-17 Common Stock $JPM $564.5M $156.76 3,601,186 27.7M"
15707,$JPM sneaky coil at 155 level 👀 https://t.co/o052Gv3RkU
10488,Master Key Trading Series #7\n\nHow to drill down for entries when Swing Trading\n\nIn this example I go through my swing trade with $XOM\n1 - Using Fibonacci on the Monthly\n2 - Using Inside Bars...
5014,$GOOGL daily... https://t.co/rpOlIUGtYY
9954,Exxon board debates dropping several major oil and gas projects - WSJ #oott $XOM https://t.co/II05AC2nKR
13208,"To Josh Brown - Totally and respectfully disagree. \nTo begin with, banks have extraordinary low/zero cost deposit bases (trillion dollar gain y/o/y) that will increase in value dramatically as i..."


### Export data

Finally let's export the processed data for EDA and modeling.

In [68]:
stocks_merged.to_csv('../data/stocks_merged.csv')